Projeto Megaline: Sugestão de Planos

A operadora de celular Megaline está insatisfeita com o fato de muitos de seus clientes estarem usando planos antigos. Ela quer desenvolver um modelo que possa analisar o comportamento do cliente e recomendar um dos planos mais recentes da Megaline: Smart ou Ultra.

Neste projeto, será desenvolvido um modelo que escolhe o plano certo de acordo com o comportamento do cliente.

1  Importação e descrição dos dados¶

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

df.info()

In [ ]:
print(df.sample(5))

2  Separação dos dados

Dividindo os dados de origem em um conjunto de treinamento, um conjunto de validação e um conjunto de teste.

In [ ]:
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra'] 

# Dividindo os dados em conjunto de treinamento e teste (75% treinamento, 25% teste)
features_train, features_test, target_train, target_test = train_test_split(
    features, target, test_size=0.25, random_state=12345)

# Dividindo os dados de treinamento em conjunto de treinamento e validação (75% treinamento, 25% validação)
features_train, features_valid, target_train, target_valid = train_test_split(
    features_train, target_train, test_size=0.25, random_state=12345)

3  Modelos Testados

3.1  Árvores de decisão¶

In [ ]:
# Definindo os intervalos para max_depth
for depth in range(1, 6):
    # Criando um modelo com a profundidade dada
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    
    # Treinando o modelo
    model.fit(features_train, target_train)
    
    # Encontrando as previsões usando o conjunto de validação
    predictions_valid = model.predict(features_valid)
    
    # Avaliando o modelo
    accuracy = accuracy_score(target_valid, predictions_valid)
    
    # Imprimindo a acurácia para a profundidade atual
    print("max_depth =", depth, ":", accuracy)

A acurácia começa a diminuir após uma profundidade máxima de 3, indicando um possível sinal de sobreajuste.

3.2  Florestas Aleatórias

In [ ]:
best_score = 0
best_est = 0

# Definindo os intervalos para n_estimators
for est in range(1, 31): 
    # Criando um modelo com o número de árvores dado
    model = RandomForestClassifier(random_state=12345, n_estimators=est)
    
    # Treinando o modelo
    model.fit(features_train, target_train)
    
    # Calculando a acurácia no conjunto de validação
    score = model.score(features_valid, target_valid)
    
    # Salvando o melhor resultado da acurácia no conjunto de validação
    if score > best_score:
        best_score = score
        best_est = est

print("Acurácia do melhor modelo no conjunto de validação (n_estimators = {}): {}".format(best_est, best_score))

O melhor modelo de Floresta Aleatória possui 20 estimadores e alcança uma acurácia de 0.83 no conjunto de validação. Isso sugere que o aumento no número de árvores na floresta levou a uma melhoria no desempenho do modelo.

3.3  Regressão Logística

In [ ]:
# Inicializando o construtor de Regressão Logística com os parâmetros fornecidos
model = LogisticRegression(random_state=12345, solver='liblinear')

# Treinando o modelo com os dados de treinamento
model.fit(features_train, target_train)

# Calculando a acurácia no conjunto de treinamento
score_train = model.score(features_train, target_train)

# Calculando a acurácia no conjunto de validação
score_valid = model.score(features_valid, target_valid)

# Imprimindo a acurácia do modelo nos conjuntos de treinamento e validação
print("Acurácia do modelo de regressão logística no conjunto de treinamento:", score_train)
print("Acurácia do modelo de regressão logística no conjunto de validação:", score_valid)

A acurácia no conjunto de validação é de 0.68. Isso indica que o modelo de Regressão Logística está generalizando bem para os dados de validação.

3.4  Qualidade do modelo escolhido

Considerando os resultados, o modelo de Florestas Aleatórias com 20 estimadores parece ter o melhor desempenho geral, pois alcança a maior acurácia no conjunto de validação. Este então é o modelo escolhido para se verificar a qualidade, treinando o modelo com o conjunto de treinamento expandido para incluir os dados de validação antes de testá-lo no conjunto de teste.

In [ ]:
# Instanciando o modelo com o melhor número de estimadores
best_model = RandomForestClassifier(random_state=54321, n_estimators=12)

# Treinando o modelo com o conjunto de treinamento completo (features_train + features_valid) e target_train
best_model.fit(features_train.append(features_valid), target_train.append(target_valid))

# Fazendo previsões usando o conjunto de teste
predictions_test = best_model.predict(features_test)

# Calculando a acurácia no conjunto de teste
accuracy_test = accuracy_score(target_test, predictions_test)

# Imprimindo a acurácia do modelo no conjunto de teste
print("Acurácia do modelo no conjunto de teste:", accuracy_test)

A acurácia do modelo no conjunto de validação foi de 0.83 com 20 estimadores, enquanto no conjunto de teste foi de 0.78. A ligeira queda na acurácia no conjunto de teste em relação ao conjunto de validação sugere que o modelo pode estar ligeiramente sobreajustado aos dados de treinamento e validação, mas ainda generaliza bem para dados não vistos.

3.5  Prova real

Treinando o modelo apenas com os dados de treinamento originais antes de testá-lo no conjunto de teste.

In [ ]:
# Instanciando o melhor modelo com o número ideal de estimadores
best_model = RandomForestClassifier(random_state=54321, n_estimators=20)

# Treinando o modelo com todos os dados de treinamento disponíveis
best_model.fit(features_train, target_train)

# Fazendo previsões nos dados de teste
predictions_test = best_model.predict(features_test)

# Calculando a acurácia do modelo nos dados de teste
accuracy_test = accuracy_score(target_test, predictions_test)

# Imprimindo a acurácia do modelo nos dados de teste
print("Acurácia do modelo no conjunto de teste:", accuracy_test)

A acurácia do modelo no conjunto de teste foi de aproximadamente 0.785, o que significa que o modelo classificou corretamente cerca de 78.5% das amostras no conjunto de teste. Isso sugere que o modelo tem uma boa capacidade de generalização para dados não vistos, pois a acurácia é alta e próxima à acurácia obtida durante a validação. Isso indica que o modelo não está sofrendo de sobreajuste aos dados de treinamento e validação e é capaz de realizar previsões precisas em novos dados. Em resumo, o modelo parece ser eficaz e estável, com um bom desempenho em dados de teste independentes.

4  Conclusões

Após avaliar e comparar diferentes modelos de aprendizado de máquina, encontramos que o modelo de Florestas Aleatórias com 20 estimadores apresentou o melhor desempenho, com uma acurácia de aproximadamente 82.8% no conjunto de validação. Esse modelo foi então testado no conjunto de teste e obteve uma acurácia de aproximadamente 78.5%, indicando que ele é capaz de generalizar bem para novos dados.

Com base no modelo selecionado, a Megaline pode utilizar as características comportamentais dos clientes para recomendar planos mais adequados (Smart ou Ultra). Isso pode ajudar a aumentar a satisfação do cliente e reduzir a rotatividade, uma vez que os clientes receberão planos que melhor atendam às suas necessidades individuais.